# Exercícios

Sua equipe  está trabalhando na auditoria da cota parlamentar para o período de 2008 a 2019. O objetivo final da sua análise é apontar possíveis irregularidades nos reembolsos, identificar solicitações suspeitas, deputados que devem ser investigados e partidos com maior índice de irregularidades. Construa uma análise utilizando dados e justificativas plausíveis possa ser utilizada na auditoria.

Escolha **três** categorias de cota parlamentar para realizar a análise. A narrativa com os dados e pensamento analítico sobre a categoria e os valores fazem parte da avaliação, que devem responder obrigatoriamente as seguintes perguntas **para cada categoria** analisada:

* Quais solicitações de reembolso são suspeitas ?
* Das solicitações suspeitas, escolha uma para analisar mais afundo e justifique o porque devemos indiciar o parlamentar ?
* Quais parlamentares devem ser investigados (top 5)?
* Eleja um parlamentar para ser premiado por boa conduta. Justifique sua escolha com dados. 

Escolha **um** parlamentar de sua preferência (alguém que você votou se lembrar). Analise os gastos específicos desse parlamentar e responda às seguintes perguntas:

* O parlamentar gasta a verba completa ou tem um gasto moderado ?
* Qual o perfil de gastos do parlamentar, quais categorias o parlamentar mais gasta a verba ?
* Existe algum reembolso suspeito em relação às categorias analisadas ?
* Analisando as empresas prestadoras de serviços (cnpj_cpf), existe algo suspeito ? 

**TENTE APLICAR A METODOLOGIA CRISP-DM**

# Iniciando a Análise

## Leitura do Dataset

In [2]:
import pandas as pd

In [3]:
dfCotaParlamentar = pd.read_csv('dadosBrutos\CEAPS_2008_2019.csv')

In [4]:
dfCotaParlamentar

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
0,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,386.60
2,2008,10,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,2610.68
4,2008,11,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
...,...,...,...,...,...,...,...,...,...,...
268894,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,O954UL,19/01/2019,"Companhia Aérea: AZUL, Localizador: O954UL. Pa...",769.29
268895,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,G9Z36V,24/01/2019,"Companhia Aérea: AZUL, Localizador: G9Z36V. Pa...",310.62
268896,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,WNQI4A,29/01/2019,"Companhia Aérea: GOL, Localizador: WNQI4A. Pas...",1168.10
268897,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",33.937.681/0001-78,LATAM,NCCPIM,23/01/2019,"Companhia Aérea: LATAM, Localizador: NCCPIM. P...",772.05


In [5]:
dfCotaParlamentar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268899 entries, 0 to 268898
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                268899 non-null  int64  
 1   MES                268899 non-null  int64  
 2   SENADOR            268899 non-null  object 
 3   TIPO_DESPESA       268899 non-null  object 
 4   CNPJ_CPF           265755 non-null  object 
 5   FORNECEDOR         265755 non-null  object 
 6   DOCUMENTO          246750 non-null  object 
 7   DATA               265755 non-null  object 
 8   DETALHAMENTO       154368 non-null  object 
 9   VALOR_REEMBOLSADO  268897 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 20.5+ MB


In [7]:
dfCotaParlamentar.sample(10)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
233079,2018,1,JOÃO CAPIBERIBE,"Locomoção, hospedagem, alimentação, combustíve...",00.521.294/0001-05,COOBRÁS RÁDIO TÁXI,24529,14/01/2018,Despesa com serviço de táxi para o Senador.,40.00
249943,2019,3,CHICO RODRIGUES,"Aluguel de imóveis para escritório político, c...",33.000.118/0008-45,Telemar Norte Leste S/A,0400007092995,22/03/2019,NaN,259.36
31663,2010,8,MARISA SERRANO,"Locomoção, hospedagem, alimentação, combustíve...",02.399.303/0001-54,Restaurante e Lanchonete fogão de Pedra,40821,24/08/2010,NaN,52.28
227934,2018,5,EDUARDO AMORIM,"Locomoção, hospedagem, alimentação, combustíve...",16.194.581/0001-80,Posto São Carlos Ltda,234962,03/05/2018,"Referente a despesa com 59,674 litros de óleo ...",226.70
260796,2019,7,MARCELO CASTRO,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,RLIJQJ,22/04/2019,"Companhia Aérea: GOL, Localizador: RLIJQJ. Pas...",974.79
79676,2012,5,LÍDICE DA MATA,"Locomoção, hospedagem, alimentação, combustíve...",177.100.145-34,Juarez Ferreira de Araújo,s/n,23/05/2012,NaN,15.00
159023,2015,10,LINDBERGH FARIAS,"Passagens aéreas, aquáticas e terrestres nacio...",16.978.175/0001-08,ADRIA Viagens e Turismo,NaN,27/10/2015,"Companhia Aérea: TAM, Localizador: 47VHF5. Pas...",872.87
28348,2010,3,JOSÉ NERY,"Locomoção, hospedagem, alimentação, combustíve...",04.780.565/0004-32,E. Carvalho Com. e Navegação Ltda.,18551,01/03/2010,NaN,114.61
249136,2019,1,ARMANDO MONTEIRO,"Passagens aéreas, aquáticas e terrestres nacio...",24.929.614/0001-10,Miranda Turismo e Rep. Ltda (Agm Turismo),358615,15/01/2019,"Companhia Aérea: GOL, Localizador: UHGGXV. Pas...",1034.58
58432,2011,8,RICARDO FERRAÇO,"Aluguel de imóveis para escritório político, c...",32.459.547/0001-46,Ilha dos Móveis LTDA. ME,0097,16/08/2011,NaN,3200.00


In [8]:
dfCotaParlamentar['VALOR_REEMBOLSADO'].iloc[0]

12351.52

In [9]:
dfCotaParlamentar.describe()

,ANO,MES,VALOR_REEMBOLSADO
count,268899.000000,268899.000000,268897.000000
mean,2014.099413,6.651040,930.433800
std,3.022301,3.282404,2453.968535
min,2008.000000,1.000000,-243.400000
25%,2012.000000,4.000000,99.700000
50%,2014.000000,7.000000,251.970000
75%,2017.000000,9.000000,900.000000
max,2019.000000,12.000000,270000.000000
